# Jupyter Widgets

Convert ipynb to PDF: This app is completely working in Jupyter but not if rendered by Voila. 

In [5]:
from ipywidgets import FileUpload, Button, Output
from IPython.display import display, FileLink, HTML
import subprocess, os


env = 'voila'   # change to 'voila' before deploying

output = Output()
fileInput = FileUpload()
convertButton = Button(description='Convert')

def convert(b):
    with output:
        print('Your PDF will be ready in a few seconds. ')
        nbName = list(fileInput.value.keys())[0]
        pdfName = nbName.replace('ipynb', 'pdf')
        with open(nbName, 'w+b') as f:
            f.write(fileInput.data[0]) 
        proc = subprocess.run(['jupyter', 'nbconvert', nbName, '--to', 'pdf'])
        if proc.returncode == 1:
            print('LaTeX compiling error!')
        else:
            pdfPath = '/'.join(['tree'] + os.getcwd().split('/')[3:] + [pdfName])
            display(HTML('''
                        <html>
                        <script>
                            var server = window.location.href;
                            var homeIdx = server.indexOf("{env}");
                            server = server.slice(0, homeIdx);
                            alert(server + "{pdfPath}");
                            document.form.link.href = server + "{pdfPath}";
                        </script>
                        <form name="form">
                            <a name="link" download>
                                <button class="p-Widget jupyter-widgets jupyter-button widget-button">Download PDF</button>
                            </a>
                        </form>
                        </html>
                         '''.format(pdfPath=pdfPath, env=env)))

convertButton.on_click(convert)

display(fileInput, convertButton, output) 

FileUpload(value={}, description='Upload')

Button(description='Convert', style=ButtonStyle())

Output()

In [ ]:
#             "https://hub.gke2.mybinder.org/user/beginnersc-sandbox-stable-cuvkwhsn/tree/misc/docs/source/ds/__dash1.pdf"
#             print('PDF is ready! You may now download it through the below link: ')
#             display(HTML("<script>alert(window.location.href);</script>"))
#             print(FileLink(pdfName))


In [9]:
# from IPython.display import Javascript
# import numpy as np

# np.array(dir(Javascript('window.location.href')))

array(['__class__', '__delattr__', '__dict__', '__dir__', '__doc__',
       '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__',
       '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__',
       '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__',
       '__repr__', '__setattr__', '__sizeof__', '__str__',
       '__subclasshook__', '__weakref__', '_check_data',
       '_data_and_metadata', '_read_flags', '_repr_javascript_',
       '_show_mem_addr', 'css', 'data', 'filename', 'lib', 'metadata',
       'reload', 'url'], dtype='<U18')